# Next.js 개발을 위한 필수 기본 지식

Next.js는 React 기반의 풀스택 웹 프레임워크로, 서버 사이드 렌더링(SSR), 정적 사이트 생성(SSG), API 라우트 등을 제공합니다.

## 1. Next.js 핵심 개념

### 1.1 페이지 기반 라우팅 시스템
- `pages` 디렉토리 구조가 자동으로 라우트 생성
- `pages/about.js` → `/about` 경로로 자동 매핑
- 동적 라우트: `pages/posts/[id].js` → `/posts/1`, `/posts/2` 등

### 1.2 렌더링 방식
- **SSG (Static Site Generation)**: 빌드 시 페이지 생성
- **SSR (Server-Side Rendering)**: 요청 시 서버에서 페이지 생성
- **CSR (Client-Side Rendering)**: 클라이언트 측에서 렌더링
- **ISR (Incremental Static Regeneration)**: 재빌드 없이 정적 페이지 업데이트

## 2. 프로젝트 구조

```
my-next-app/
├── pages/              # 라우트 페이지
│   ├── api/            # API 라우트
│   ├── _app.js         # 공통 레이아웃
│   └── index.js        # 홈페이지
├── public/             # 정적 파일
├── styles/             # CSS 모듈
├── components/         # 재사용 컴포넌트
├── lib/                # 유틸리티 함수
├── next.config.js      # Next.js 설정
└── package.json
```

## 3. 필수 API

### 3.1 데이터 가져오기
```javascript
// SSG (빌드 시 데이터 가져오기)
export async function getStaticProps() {
  const res = await fetch('https://api.example.com/data');
  const data = await res.json();
  return { props: { data } };
}

// SSR (요청 시 데이터 가져오기)
export async function getServerSideProps(context) {
  const { params, req, res } = context;
  const data = await fetchData(params.id);
  return { props: { data } };
}
```

### 3.2 API 라우트
```javascript
// pages/api/hello.js
export default function handler(req, res) {
  res.status(200).json({ name: 'John Doe' });
}
```

## 4. 주요 기능

### 4.1 이미지 최적화
```javascript
import Image from 'next/image';

<Image
  src="/profile.jpg"
  alt="Profile"
  width={500}
  height={500}
  priority // 중요 이미지 표시
/>
```

### 4.2 동적 임포트 (코드 스플리팅)
```javascript
import dynamic from 'next/dynamic';

const DynamicComponent = dynamic(
  () => import('../components/HeavyComponent'),
  { loading: () => <p>Loading...</p>, ssr: false }
);
```

### 4.3 SEO 최적화
```javascript
import Head from 'next/head';

function HomePage() {
  return (
    <>
      <Head>
        <title>My Page Title</title>
        <meta name="description" content="My page description" />
      </Head>
      {/* 페이지 내용 */}
    </>
  );
}
```

## 5. 스타일링 방법

### 5.1 CSS 모듈
```javascript
// styles/Button.module.css
.button {
  background: blue;
}

// components/Button.js
import styles from '../styles/Button.module.css';

export default function Button() {
  return <button className={styles.button}>Click</button>;
}
```

### 5.2 Tailwind CSS (권장)
```javascript
// 설치 후 사용
<button className="bg-blue-500 hover:bg-blue-700 text-white font-bold py-2 px-4 rounded">
  Button
</button>
```

### 5.3 styled-jsx (Next.js 기본)
```javascript
<style jsx>{`
  .container {
    margin: 0 auto;
    max-width: 1200px;
  }
`}</style>
```

## 6. 환경 변수 관리

```javascript
// .env.local
NEXT_PUBLIC_API_URL=https://api.example.com
SECRET_KEY=mysecretkey

// 사용 시
const apiUrl = process.env.NEXT_PUBLIC_API_URL; // 클라이언트 접근 가능
const secret = process.env.SECRET_KEY; // 서버에서만 접근 가능
```

## 7. 인증 구현 패턴

### 7.1 JWT 기반 인증
```javascript
// API 라우트 예제 (pages/api/login.js)
import jwt from 'jsonwebtoken';

export default function handler(req, res) {
  if (req.method === 'POST') {
    const token = jwt.sign({ userId: 123 }, process.env.JWT_SECRET, {
      expiresIn: '1h',
    });
    res.setHeader('Set-Cookie', `token=${token}; HttpOnly; Path=/`);
    res.status(200).json({ success: true });
  }
}
```

### 7.2 NextAuth.js (추천)
```javascript
// pages/api/auth/[...nextauth].js
import NextAuth from 'next-auth';
import Providers from 'next-auth/providers';

export default NextAuth({
  providers: [
    Providers.GitHub({
      clientId: process.env.GITHUB_ID,
      clientSecret: process.env.GITHUB_SECRET,
    }),
  ],
});
```

## 8. 배포 전략

### 8.1 Vercel (공식 호스팅)
- 자동 CI/CD
- 프리티어로 시작 가능
- 서버리스 함수 포함

### 8.2 다른 플랫폼
- Dockerize 후 배포
- Node.js 서버에서 실행 (`next start`)

## 9. 성능 최적화 기법

1. **정적 자산 최적화**: Image 컴포넌트 사용
2. **코드 스플리팅**: 동적 임포트 활용
3. **프리페칭**: `<Link prefetch>` 사용
4. **ISR**: 자주 변경되는 데이터에 적용
5. **에지 함수**: 지연 시간 감소

## 10. 테스팅

```javascript
// Jest + React Testing Library 예제
import { render, screen } from '@testing-library/react';
import Home from '../pages/index';

describe('Home', () => {
  it('renders heading', () => {
    render(<Home />);
    expect(screen.getByRole('heading')).toHaveTextContent('Welcome');
  });
});
```

## 11. TypeScript 통합

```javascript
// 타입이 적용된 페이지 예제
import { NextPage } from 'next';

interface Props {
  user: {
    name: string;
    age: number;
  };
}

const UserPage: NextPage<Props> = ({ user }) => {
  return <div>{user.name}</div>;
};

export default UserPage;
```

Next.js는 React 생태계의 최신 기능들을 통합하면서도 개발자 경험을 크게 향상시킨 프레임워크입니다. 위의 핵심 개념들을 숙지한 후, 점진적으로 App Router, Server Components, Streaming 등 최신 기능들을 학습하는 것을 추천합니다.